<a href="https://colab.research.google.com/github/cvs2010/rsp/blob/main/notebook_teste_importsheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook criado com intuito de gerar um algoritmo para importação dos dados das planilhas Horario RSP

Observações:
- Tente o mínimo possível utilizar as bandas de GPU e TPU grátis do colab
- Esse arquivo é um arquivo de histórico, então tente trabalhar no seu GoogleColab em paralelo, somente suba as células para esse Notebook após finalizar todo o código ou gerar um checkpoint de análise do código (momento em que o código ainda não está pronto, mas necessita de análise do grupo)
- Muito cuidado com a edição desse arquivo e de outros arquivos da pasta do Drive: mais uma vez, tente trabalhar em paralelo, nunca utilize os arquivos originais, faça cópia de arquivos e trabalhe com seus arquivos localmente ou via cloud em seu login.

Importação de bibliotecas

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import numpy as np
import os